## 🐍 Python 텍스트 분석: 고전적 벡터화 기법 마스터하기
이번 시간에는 자연어 처리(NLP)의 가장 기본적이면서도 중요한 단계인 **텍스트 벡터화(Text Vectorization)** 의 고전적인 기법들을 다룹니다. 

컴퓨터가 텍스트를 이해하고 처리할 수 있도록 숫자로 변환하는 다양한 방법을 배우고, `kiwipiepy`를 활용한 한국어 처리 예제를 통해 실전 감각을 익힙니다.

---

### 1. Bag-of-Words (BoW)와 문서-단어 행렬 (DTM)

#### 💡 개념 (Concept)

머신러닝 모델은 숫자 데이터만 처리할 수 있으므로, 텍스트를 숫자 벡터로 변환하는 과정이 필수적입니다. 

**Bag-of-Words(BoW)** 는 가장 기초적인 텍스트 표현 방법으로, 이름처럼 문서를 '단어들의 가방'으로 간주합니다. 

이 모델은 문맥이나 단어의 순서는 무시하고, 각 단어가 문서에 몇 번 등장했는지(빈도)에만 집중합니다.

BoW 모델을 사용해 텍스트 모음(Corpus)을 벡터화하면 **문서-단어 행렬(Document-Term Matrix, DTM)** 이 생성됩니다. DTM에서 각 행은 문서를, 각 열은 전체 어휘 사전에 포함된 단어를, 그리고 각 셀의 값은 해당 단어가 해당 문서에 나타난 빈도를 의미합니다.

#### 💻 예시 코드 (Example Code)

`scikit-learn`의 `CountVectorizer`는 BoW 모델을 손쉽게 구현하도록 도와줍니다. 한국어는 조사가 발달하여 "영화가", "영화는" 등을 다른 단어로 인식하는 문제가 있으므로, `kiwipiepy` 형태소 분석기를 토크나이저로 연결하여 사용해야 합니다.

In [2]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer

# 1. kiwipiepy 형태소 분석기 준비
kiwi = Kiwi()

# 2. CountVectorizer에 연결할 토크나이저 함수 정의
# 명사(NNG, NNP), 동사 어근(VV), 형용사 어근(VA)만 추출
def kiwi_tokenizer(text: str) -> list[str]:
    """kiwipiepy를 사용하여 명사, 동사, 형용사를 추출하는 토크나이저"""
    tokens = kiwi.tokenize(text) # 리스트 형식으로 토큰이 나오게 해줘야함 - callback 함수
    return [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA']]


In [3]:
# 3. 실습용 텍스트 데이터
corpus = [
    '배우의 연기력이 정말 대단한 영화였어요.',
    '스토리가 너무 예측 가능해서 연기력이 아까웠다.',
    '감독의 연출과 배우의 연기가 조화로웠던 영화.',
]

# 4. CountVectorizer 생성 및 DTM 구축
vectorizer = CountVectorizer(tokenizer=kiwi_tokenizer)
dtm = vectorizer.fit_transform(corpus) # fit(corpus).transform(corpus)를 한 번에 해주는 함수

# CSR : 압축해서 저장함 (빈 공간 없이 저장 후 압축)

# 생성된 어휘 사전 확인
feature_names = vectorizer.get_feature_names_out()
print("어휘 사전 (Vocabulary):")
print(feature_names)

# DTM을 DataFrame으로 시각화
dtm_df = pd.DataFrame(dtm.toarray(), columns=feature_names)
print("\n문서-단어 행렬 (DTM):")
print(dtm_df)

# 압축된 dtm을 풀려면 dtm.toarray()하면 됨

어휘 사전 (Vocabulary):
['가능' '감독' '배우' '스토리' '연기' '연기력' '연출' '영화' '예측' '조화']

문서-단어 행렬 (DTM):
   가능  감독  배우  스토리  연기  연기력  연출  영화  예측  조화
0   0   0   1    0   0    1   0   1   0   0
1   1   0   0    1   0    1   0   0   1   0
2   0   1   1    0   1    0   1   1   0   1


c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#### ✏️ 연습 문제 (Practice Problems)

1.  위 예제의 `kiwi_tokenizer` 함수를 수정하여, 품사가 `NNG`(일반 명사)인 단어만 추출하도록 변경한 뒤 DTM을 다시 생성하고 결과를 비교해 보세요.


In [4]:
# 코드 작성


2.  `CountVectorizer`를 생성할 때 `ngram_range=(1, 2)` 파라미터를 추가하여 DTM을 만들어 보세요. 생성된 어휘 사전에 어떤 변화가 생겼는지, 그리고 이 파라미터가 어떤 의미를 갖는지 설명해 보세요.

In [5]:
# 코드 작성

---

### 🚀 여기서 잠깐! Python 실력 다지기: 좋은 디자인 패턴 클래스로 래핑하기

위의 코드는 간단한 예시였지만, 실제 프로젝트에서는 더 체계적이고 재사용 가능한 코드 구조가 필요합니다. 

아래에서는 **싱글톤 패턴(Singleton Pattern)** 을 활용하여 Kiwi 형태소 분석기를 효율적으로 관리하고, 텍스트 마이닝 작업을 캡슐화한 클래스를 만들어보겠습니다.

**주요 개선 사항:**
- 🎯 **싱글톤 패턴**: Kiwi 객체를 한 번만 생성하여 메모리 효율성 증대
- 🔧 **캡슐화**: 토큰화와 DTM 생성 기능을 하나의 클래스로 통합
- 🎨 **유연성**: 품사 태그를 매개변수로 받아 다양한 분석 요구사항 대응
- 📊 **편의성**: DTM을 바로 DataFrame 형태로 반환하여 분석 작업 간소화

In [6]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer


# __new__ : 객체를 생성할 때마다 딱 한 번만 실행 (인스턴스가 여러개 발급 되더라도 한 번만 실행) -> 한 번만 남는다고 해서 싱글톤 패턴이라고 함
# kiwitoken = KiwiTokenizer() 

# 기능을 강화해 새로운 class 생성 = wrapping
class KiwiTokenizer:
    """Kiwi 형태소 분석기를 싱글톤 패턴으로 관리하는 토크나이저 클래스"""
    # kiwi라는 인스턴스를 반복적으로 만들지 않도록 해줌
    
    _instance = None
    _kiwi = None
    
    def __new__(cls): # __new__ 메서드는 클래스 인스턴스를 생성할 때 호출되는 메서드입니다.
        if cls._instance is None:
            cls._instance = super(KiwiTokenizer, cls).__new__(cls)
            cls._kiwi = Kiwi() # Kiwi 객체 생성
            cls._instance.vectorizer = None  # vectorizer 초기화
        return cls._instance
    
    def tokenize(self, text: str, pos_tags: list[str] = None) -> list[str]:
        """
        텍스트를 토큰화하여 지정된 품사의 단어만 추출
        
        Args:
            text: 분석할 텍스트
            pos_tags: 추출할 품사 태그 리스트 (기본값: ['NNG', 'NNP', 'VV', 'VA'])
        
        Returns:
            추출된 단어들의 리스트
        """
        if pos_tags is None:
            pos_tags = ['NNG', 'NNP', 'VV', 'VA']
        
        tokens = self._kiwi.tokenize(text)
        return [token.form for token in tokens if token.tag in pos_tags]
    
    def create_dtm(self, corpus: list[str], pos_tags: list[str] = None) -> pd.DataFrame:
        """
        텍스트 코퍼스를 이용하여 DTM(Document-Term Matrix)을 생성
        
        Args:
            corpus: 텍스트 문서들의 리스트
            pos_tags: 추출할 품사 태그 리스트 (기본값: ['NNG', 'NNP', 'VV', 'VA'])
        
        Returns:
            DTM DataFrame
        """
        def tokenizer_func(text: str) -> list[str]:
            return self.tokenize(text, pos_tags)
        
        # class 내에 일단 저장 (class 내에서 저장 시 self. 이용)
        self.vectorizer = CountVectorizer(tokenizer=tokenizer_func)
        dtm = self.vectorizer.fit_transform(corpus)
        feature_names = self.vectorizer.get_feature_names_out()
        dtm_df = pd.DataFrame(dtm.toarray(), columns=feature_names)
        
        return dtm_df
    
    def get_vocabulary(self) -> list[str]:
        """생성된 어휘 사전을 반환"""
        if self.vectorizer is None:
            print("먼저 create_dtm() 메서드를 호출하여 vectorizer를 생성해주세요.")
            return
        feature_names = self.vectorizer.get_feature_names_out()
        return list(feature_names)

In [7]:
# 토크나이저 인스턴스 생성
tokenizer = KiwiTokenizer()
# tokenizer1 = KiwiTokenizer() # kiwi 인스턴스는 한 번만 실행

In [8]:
dtm_df = tokenizer.create_dtm(corpus)
dtm_df

c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,가능,감독,배우,스토리,연기,연기력,연출,영화,예측,조화
0,0,0,1,0,0,1,0,1,0,0
1,1,0,0,1,0,1,0,0,1,0
2,0,1,1,0,1,0,1,1,0,1


In [9]:
tokenizer.get_vocabulary()

['가능', '감독', '배우', '스토리', '연기', '연기력', '연출', '영화', '예측', '조화']

-----

### 2\. TF-IDF (Term Frequency-Inverse Document Frequency)

#### 💡 개념 (Concept)

DTM은 구현이 간단하지만, "그리고", "있다"와 같이 모든 문서에 자주 나타나는 단어들이 높은 값을 가져 중요도를 왜곡할 수 있다는 한계가 있습니다. **TF-IDF**는 이러한 단점을 보완하기 위해 등장한 가중치 부여 방식입니다.

  - **TF (단어 빈도, Term Frequency)**: 특정 문서 내에서 단어가 얼마나 자주 등장하는지를 나타내는 값. DTM의 값과 같습니다.
  - **IDF (역문서 빈도, Inverse Document Frequency)**: 특정 단어가 전체 문서에서 얼마나 희귀하게 등장하는지를 나타내는 값입니다. 모든 문서에 흔하게 등장하는 단어는 낮은 IDF 값을, 특정 소수의 문서에만 등장하는 희귀한 단어는 높은 IDF 값을 갖습니다.

**TF-IDF 값은 TF와 IDF를 곱하여 계산**되며, 특정 문서에는 자주 등장하지만(높은 TF) 전체 문서에서는 희귀한(높은 IDF) 단어일수록 높은 값을 가집니다. 이는 그 단어가 해당 문서를 대표하는 **핵심어**일 가능성이 높다는 의미입니다.

![수식](https://blog.kakaocdn.net/dn/QOLOh/btrFbMBsp6x/wln93zToLdJS6QR9g1Kwg0/img.png)

In [10]:
# corpus : 문장이 여러개 == 전체 문서
# 문장 == 문서
# 덜 중요한지를 판단하기 위해서 (자주 등장은 하지만 중요하지 않은 단어)
# 예) '나는'
# 많은 검색 엔진들이 tf-idf계열 사용 -> 의미 있게 많이 등장해야 검색이 됨


#### 💻 예시 코드 (Example Code)
`scikit-learn`의 `TfidfVectorizer`를 사용하면 TF-IDF 행렬을 쉽게 생성할 수 있습니다.


In [11]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. kiwipiepy 토크나이저 (위와 동일)
kiwi = Kiwi()
def kiwi_tokenizer(text: str) -> list[str]:
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA']]

# 2. 실습용 텍스트 데이터
corpus = [
    '배우의 연기력이 정말 대단한 영화였어요.',
    '스토리가 너무 예측 가능해서 연기력이 아까웠다.',
    '감독의 연출과 배우의 연기가 조화로웠던 영화.',
    '와 이 영화 진짜 대박이야! 배우들 연기 미쳤고 스토리도 완전 몰입됨',
    '음... 좀 아쉽네요. 감독이 뭘 말하고 싶었는지 모르겠어요',
    '연기는 괜찮았는데 결말이 너무 뻔해서 실망했습니다',
    '헐 이거 완전 꿀잼ㅋㅋ 예상 못한 반전에 소름돋았어',
    '감독님... 제발 좀 더 신경써서 찍으시길... 연출이 엉망이에요',
    '주연배우 연기 진짜 자연스럽더라! 몰입도 최고였음',
    '스토리가 조금 복잡하긴 했지만 나름 볼만했어요',
    '이런 영화를 왜 만들었는지 이해가 안 가네... 시간 아까움',
    '배우들 케미 완전 좋았고 연출도 깔끔했음. 추천!',
    '예측할 수 없는 전개로 끝까지 출긴장감 넘쳤습니다',
    '연기력은 인정하지만 스토리가 너무 뻔해서... 그냥 그래요',
    '감독의 의도는 좋았으나 표현 방식이 아쉬웠네요',
    'ㅋㅋㅋ 이거 뭐야 완전 재밌잖아? 배우들 연기 ㄹㅇ 대단함',
    '조용한 영화인데 배우들 연기가 워낙 좋아서 지루하지 않았어요',
    '액션은 별로였지만 인간관계 드라마가 탄탄해서 만족',
    '아 진짜... 왜 이렇게 만들었을까? 감독 뭐하는 거야',
    '처음엔 지루했는데 중반부터 완전 몰입! 연출 센스 있네',
    '배우들 연기는 좋았지만 전체적으로 밋밋한 느낌이에요',
    '와... 이런 스토리는 처음 봐! 정말 신선하고 감동적이었어',
    '연출과 연기 모두 완벽했습니다. 올해 최고의 작품 중 하나!',
    '뭔가 아쉬운 부분들이 있지만 그래도 볼만한 영화였어요'
]

# 3. TfidfVectorizer 생성 및 TF-IDF 행렬 구축
tfidf_vectorizer = TfidfVectorizer(tokenizer=kiwi_tokenizer)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus) # 적합과 동시에 변환

# csr matrix : csr 압축된 매트릭스

c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
# 대용량 데이터일 시 메모리가 꽉 차버림 - df 만들기 x

# 어휘 사전 확인
feature_names = tfidf_vectorizer.get_feature_names_out()

# TF-IDF 행렬을 DataFrame으로 시각화 (소수점 셋째 자리까지)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df.round(3)

,가,가능,감독,감동,결말,괜찮,긴장감,깔끔,꿀,끝,...,주연,중반,찍,처음,최고,추천,출,케미,표현,하
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.587,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.396,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.438,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.493,0.493,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.421,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.338
7,0.000,0.000,0.309,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.45,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.475,0.000,0.00,0.000,0.420,0.000,0.000,0.000,0.000,0.000
9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.582


#### ✏️ 연습 문제 (Practice Problems)

1.  학습이 완료된 `tfidf_vectorizer` 객체의 `idf_` 속성을 출력해 보세요. 어떤 단어의 IDF 값이 가장 높고, 어떤 단어의 IDF 값이 가장 낮은가요? 그 이유를 설명해 보세요. (힌트: `zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)`를 사용하면 단어와 IDF 값을 함께 볼 수 있습니다.)

In [13]:
pd.DataFrame(list(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)), columns=['단어', 'IDF'])

,단어,IDF
0,가,3.525729
1,가능,3.525729
2,감독,2.427116
3,감동,3.525729
4,결말,3.525729
...,...,...
64,추천,3.525729
65,출,3.525729
66,케미,3.525729
67,표현,3.525729


2.  `TfidfVectorizer` 생성 시 `max_df=0.8`과 `min_df=2` 파라미터를 추가하여 TF-IDF 행렬을 다시 만들고, 어휘 사전이 어떻게 변하는지 확인해 보세요. 각 파라미터가 어떤 역할을 하는지 설명해 보세요.


In [14]:
# 코드 생성
tfidf_vectorizer2 = TfidfVectorizer(tokenizer=kiwi_tokenizer, max_df=2, min_df=1)
tfidf_matrix2 = tfidf_vectorizer2.fit_transform(corpus)

pd.DataFrame(tfidf_matrix2.toarray(), columns=tfidf_vectorizer2.get_feature_names_out())

c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,가,가능,감동,결말,괜찮,긴장감,깔끔,꿀,끝,넘치,...,조화,주연,중반,찍,처음,최고,추천,출,케미,표현
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
1,0.000000,0.748854,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
5,0.000000,0.000000,0.000000,0.514125,0.514125,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.447214,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.5,0.000000,0.000,0.00000,0.000000,0.00000,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.514125,0.000000,0.0,0.000000,0.455,0.00000,0.000000,0.00000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.00000,0.000000,0.00000,0.00000


## 3. BM25 알고리즘 (Best Matching 25)

**BM25(Best Matching 25)** 는 정보 검색 분야에서 문서와 질의(query) 간의 관련성을 평가하는 대표적인 랭킹 함수입니다. 

TF-IDF에서 더 발전된 알고리즘으로, 문서 길이와 단어길이를 보정하여 더 정규화된 점수를 계산합니다.

검색 엔진(예: 구글, 네이버, 엘라스틱서치 등)에서 사용자가 입력한 키워드와 가장 관련성이 높은 문서를 찾아 순위를 매기는 데 널리 활용됩니다.


### BM25의 핵심 원리

- **TF(단어 빈도, Term Frequency):** 특정 단어가 문서 내에서 얼마나 자주 등장하는지 측정합니다.
- **IDF(역문서 빈도, Inverse Document Frequency):** 특정 단어가 전체 문서 집합에서 얼마나 희귀한지 반영합니다. 자주 등장하지 않는 단어일수록 더 중요한 정보로 간주합니다.
- **문서 길이 보정:** 문서마다 길이가 다르기 때문에, 긴 문서가 점수를 과도하게 받지 않도록 길이 보정이 들어갑니다.

BM25는 TF-IDF와 비슷한 개념을 사용하지만, TF(단어 빈도)의 증가에 따라 점수가 무한정 커지지 않도록 조절하고, 문서 길이 차이도 공정하게 반영합니다.

### BM25의 수식

BM25 점수는 다음과 같이 계산됩니다.

$$
\text{Score}(D, Q) = \sum_{q_i \in Q} \text{IDF}(q_i) \cdot \frac{TF(q_i, D) \cdot (k_1 + 1)}{TF(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}
$$

- $TF(q_i, D)$: 문서 D에서 단어 $q_i$의 빈도
- $|D|$: 문서 D의 길이(단어 수)
- $\text{avgdl}$: 전체 문서의 평균 길이
- $k_1, b$: 조정 가능한 하이퍼파라미터(보통 $k_1$은 1.2~2.0, $b$는 0.75)

### BM25의 특징과 장점

- **정확성:** 문서와 질의 간의 관련성을 효과적으로 평가
- **유연성:** 파라미터 조정으로 다양한 데이터셋에 대응 가능
- **효율성:** 계산이 간단해 대규모 데이터셋에서도 빠르게 작동
- **검색 엔진 표준:** 엘라스틱서치, 구글, 네이버 등 주요 검색 시스템에서 기본 랭킹 함수로 채택

### BM25와 TF-IDF의 차이점

| 구분         | TF-IDF                                   | BM25                                                         |
|--------------|------------------------------------------|--------------------------------------------------------------|
| 단어 빈도    | 단순히 많이 등장할수록 점수 증가           | TF가 일정 수준 이상 올라가지 않도록 조절                      |
| 문서 길이    | 고려하지 않음                             | 문서 길이 보정 적용(긴 문서 불리, 짧은 문서 유리 현상 완화)   |
| 파라미터     | 없음                                     | k1, b 등 파라미터로 세밀한 조정 가능                         |

### BM25의 응용 분야

- 검색 엔진(웹, 문서, 뉴스 등)
- 추천 시스템
- 자연어 처리 기반 문서 분류, 요약 등

In [15]:
# 뼈대는 tf-idf, 극단으로 치우치지 않게 정규화를 해주는 값들이 들어가 있음


#### 💻 예시 코드 (Example Code)

In [16]:
!pip install rank_bm25


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from rank_bm25 import BM25Okapi

# 1. BM25 모델 생성 및 학습
print("* BM25 모델 생성 및 학습")
tokenized_corpus_for_bm25 = [kiwi_tokenizer(doc) for doc in corpus]
print("* 토큰화된 문서들")
for i, tokens in enumerate(tokenized_corpus_for_bm25):
    print(f"문서 {i}: {tokens}")

bm25 = BM25Okapi(tokenized_corpus_for_bm25)
print("* BM25 모델 학습 완료")

* BM25 모델 생성 및 학습
* 토큰화된 문서들
문서 0: ['배우', '연기력', '영화']
문서 1: ['스토리', '예측', '가능', '연기력']
문서 2: ['감독', '연출', '배우', '연기', '조화', '영화']
문서 3: ['영화', '대박', '배우', '연기', '미치', '스토리', '몰입']
문서 4: ['감독', '말', '모르']
문서 5: ['연기', '괜찮', '결말', '뻔하', '실망']
문서 6: ['꿀', '잼', '예상', '하', '반전', '소름']
문서 7: ['감독', '신경', '쓰', '찍', '연출', '엉망']
문서 8: ['주연', '배우', '연기', '자연', '몰입도', '최고']
문서 9: ['스토리', '보', '하']
문서 10: ['영화', '만들', '이해', '가', '시간']
문서 11: ['배우', '케미', '좋', '연출', '깔끔', '추천']
문서 12: ['예측', '없', '전개', '끝', '출', '긴장감', '넘치']
문서 13: ['연기력', '인정', '스토리', '뻔하']
문서 14: ['감독', '의도', '좋', '표현', '방식']
문서 15: ['재밌', '배우', '연기', '대단하']
문서 16: ['영화', '배우', '연기', '좋']
문서 17: ['액션', '인간관계', '드라마', '만족']
문서 18: ['만들', '감독', '하']
문서 19: ['처음', '중반', '몰입', '연출', '센스', '있']
문서 20: ['배우', '연기', '좋', '전체', '느낌']
문서 21: ['스토리', '처음', '보', '감동']
문서 22: ['연출', '연기', '완벽', '올해', '최고', '작품']
문서 23: ['부분', '있', '영화']
* BM25 모델 학습 완료


In [18]:
# 2. 질의(Query)에 대한 BM25 점수 계산
query = "배우 연기력"
tokenized_query = kiwi_tokenizer(query) # 질문을 할 때 사용
print(f"\n질의: '{query}'")
print(f"토큰화된 질의: {tokenized_query}")


질의: '배우 연기력'
토큰화된 질의: ['배우', '연기력']


In [19]:
# 각 문서에 대한 BM25 점수 계산
doc_scores = bm25.get_scores(tokenized_query) # transform과 비슷
print(f"각 문서의 BM25 점수:")
for i, score in enumerate(doc_scores):
    print(f"문서 {i}: {score:.4f}")

각 문서의 BM25 점수:
문서 0: 2.9800
문서 1: 1.9611
문서 2: 0.5957
문서 3: 0.5494
문서 4: 0.0000
문서 5: 0.0000
문서 6: 0.0000
문서 7: 0.0000
문서 8: 0.5957
문서 9: 0.0000
문서 10: 0.0000
문서 11: 0.5957
문서 12: 0.0000
문서 13: 1.9611
문서 14: 0.0000
문서 15: 0.7166
문서 16: 0.7166
문서 17: 0.0000
문서 18: 0.0000
문서 19: 0.0000
문서 20: 0.6506
문서 21: 0.0000
문서 22: 0.0000
문서 23: 0.0000


In [20]:
# 3. 가장 관련성 높은 문서 반환
print(f"가장 관련성 높은 문서 순위:")
# 점수와 인덱스를 함께 저장하여 정렬
score_doc_pairs = [(score, i, corpus[i]) for i, score in enumerate(doc_scores)]
sorted_results = sorted(score_doc_pairs, key=lambda x: x[0], reverse=True)

for rank, (score, doc_idx, doc_text) in enumerate(sorted_results, 1):
    print(f"{rank}위 (점수: {score:.4f}): {doc_text}")

가장 관련성 높은 문서 순위:
1위 (점수: 2.9800): 배우의 연기력이 정말 대단한 영화였어요.
2위 (점수: 1.9611): 스토리가 너무 예측 가능해서 연기력이 아까웠다.
3위 (점수: 1.9611): 연기력은 인정하지만 스토리가 너무 뻔해서... 그냥 그래요
4위 (점수: 0.7166): ㅋㅋㅋ 이거 뭐야 완전 재밌잖아? 배우들 연기 ㄹㅇ 대단함
5위 (점수: 0.7166): 조용한 영화인데 배우들 연기가 워낙 좋아서 지루하지 않았어요
6위 (점수: 0.6506): 배우들 연기는 좋았지만 전체적으로 밋밋한 느낌이에요
7위 (점수: 0.5957): 감독의 연출과 배우의 연기가 조화로웠던 영화.
8위 (점수: 0.5957): 주연배우 연기 진짜 자연스럽더라! 몰입도 최고였음
9위 (점수: 0.5957): 배우들 케미 완전 좋았고 연출도 깔끔했음. 추천!
10위 (점수: 0.5494): 와 이 영화 진짜 대박이야! 배우들 연기 미쳤고 스토리도 완전 몰입됨
11위 (점수: 0.0000): 음... 좀 아쉽네요. 감독이 뭘 말하고 싶었는지 모르겠어요
12위 (점수: 0.0000): 연기는 괜찮았는데 결말이 너무 뻔해서 실망했습니다
13위 (점수: 0.0000): 헐 이거 완전 꿀잼ㅋㅋ 예상 못한 반전에 소름돋았어
14위 (점수: 0.0000): 감독님... 제발 좀 더 신경써서 찍으시길... 연출이 엉망이에요
15위 (점수: 0.0000): 스토리가 조금 복잡하긴 했지만 나름 볼만했어요
16위 (점수: 0.0000): 이런 영화를 왜 만들었는지 이해가 안 가네... 시간 아까움
17위 (점수: 0.0000): 예측할 수 없는 전개로 끝까지 출긴장감 넘쳤습니다
18위 (점수: 0.0000): 감독의 의도는 좋았으나 표현 방식이 아쉬웠네요
19위 (점수: 0.0000): 액션은 별로였지만 인간관계 드라마가 탄탄해서 만족
20위 (점수: 0.0000): 아 진짜... 왜 이렇게 만들었을까? 감독 뭐하는 거야
21위 (점수: 0.0000): 처음엔 지루했는데 중반부

In [21]:
# 4. get_top_n 메서드 사용
top_n_docs = bm25.get_top_n(tokenized_query, corpus, n=3)
print(f"상위 3개 문서 (get_top_n 사용):")
for i, doc in enumerate(top_n_docs, 1):
    print(f"{i}. {doc}")

상위 3개 문서 (get_top_n 사용):
1. 배우의 연기력이 정말 대단한 영화였어요.
2. 스토리가 너무 예측 가능해서 연기력이 아까웠다.
3. 연기력은 인정하지만 스토리가 너무 뻔해서... 그냥 그래요


In [22]:
# 5. 다른 질의로 테스트
query2 = "스토리 감독"
tokenized_query2 = kiwi_tokenizer(query2)
print(f"질의: '{query2}'")
print(f"토큰화된 질의: {tokenized_query2}")

# doc_scores2 = bm25.get_scores(tokenized_query2)
# print(f"\n각 문서의 BM25 점수:")
# for i, score in enumerate(doc_scores2):
#     print(f"문서 {i}: {score:.4f}")

top_n_docs = bm25.get_top_n(tokenized_query2, corpus, n=3)
print(f"상위 3개 문서 (get_top_n 사용):")
for i, doc in enumerate(top_n_docs, 1):
    print(f"{i}. {doc}")  

질의: '스토리 감독'
토큰화된 질의: ['스토리', '감독']
상위 3개 문서 (get_top_n 사용):
1. 스토리가 조금 복잡하긴 했지만 나름 볼만했어요
2. 아 진짜... 왜 이렇게 만들었을까? 감독 뭐하는 거야
3. 음... 좀 아쉽네요. 감독이 뭘 말하고 싶었는지 모르겠어요


In [23]:
# 6. BM25 파라미터 조정 (k1, b 값 변경)
print("BM25 파라미터 조정 (k1=0.5, b=0.75)")

bm25_custom = BM25Okapi(tokenized_corpus_for_bm25, k1=0.5, b=0.75)
doc_scores_custom = bm25_custom.get_scores(tokenized_query)

print("기본 파라미터 vs 조정된 파라미터 비교:")
print("문서\t기본 BM25\t조정 BM25\t차이")
for i, (default_score, custom_score) in enumerate(zip(doc_scores, doc_scores_custom)):
    diff = custom_score - default_score
    print(f"{i}\t{default_score:.4f}\t\t{custom_score:.4f}\t\t{diff:+.4f}")

print("\n=== BM25 분석 ===")
print("BM25 점수가 높을수록 질의와 관련성이 높은 문서입니다.")
print("k1: 단어 빈도의 포화도 조절 (높을수록 빈도 영향 증가)")
print("b: 문서 길이 정규화 정도 (0에 가까우면 길이 무시, 1에 가까우면 길이 크게 반영)")

BM25 파라미터 조정 (k1=0.5, b=0.75)
기본 파라미터 vs 조정된 파라미터 비교:
문서	기본 BM25	조정 BM25	차이
0	2.9800		2.7342		-0.2458
1	1.9611		1.8935		-0.0676
2	0.5957		0.6240		+0.0283
3	0.5494		0.5948		+0.0454
4	0.0000		0.0000		+0.0000
5	0.0000		0.0000		+0.0000
6	0.0000		0.0000		+0.0000
7	0.0000		0.0000		+0.0000
8	0.5957		0.6240		+0.0283
9	0.0000		0.0000		+0.0000
10	0.0000		0.0000		+0.0000
11	0.5957		0.6240		+0.0283
12	0.0000		0.0000		+0.0000
13	1.9611		1.8935		-0.0676
14	0.0000		0.0000		+0.0000
15	0.7166		0.6919		-0.0247
16	0.7166		0.6919		-0.0247
17	0.0000		0.0000		+0.0000
18	0.0000		0.0000		+0.0000
19	0.0000		0.0000		+0.0000
20	0.6506		0.6562		+0.0056
21	0.0000		0.0000		+0.0000
22	0.0000		0.0000		+0.0000
23	0.0000		0.0000		+0.0000

=== BM25 분석 ===
BM25 점수가 높을수록 질의와 관련성이 높은 문서입니다.
k1: 단어 빈도의 포화도 조절 (높을수록 빈도 영향 증가)
b: 문서 길이 정규화 정도 (0에 가까우면 길이 무시, 1에 가까우면 길이 크게 반영)


In [24]:
# 7. TF-IDF와 BM25 비교
print("TF-IDF vs BM25 비교 (질의: '배우 연기력')")

# TF-IDF로 질의 변환
query_tfidf = tfidf_vectorizer.transform([query])
# 각 문서와의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
tfidf_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

print("문서\tTF-IDF 유사도\tBM25 점수\t선호 모델")
for i, (tfidf_sim, bm25_score) in enumerate(zip(tfidf_similarities, doc_scores)):
    preferred = "BM25" if bm25_score > tfidf_sim else "TF-IDF" if tfidf_sim > bm25_score else "동일"
    print(f"{i}\t{tfidf_sim:.4f}\t\t{bm25_score:.4f}\t\t{preferred}")

print("\n=== 비교 분석 ===")
print("TF-IDF: 코사인 유사도 기반, 벡터 공간에서의 각도 측정")
print("BM25: 확률론적 랭킹 함수, 문서 길이와 단어 빈도 포화 고려")
print("BM25가 일반적으로 검색 성능이 더 우수하다고 알려져 있습니다.")


TF-IDF vs BM25 비교 (질의: '배우 연기력')
문서	TF-IDF 유사도	BM25 점수	선호 모델
0	0.8372		2.9800		BM25
1	0.3838		1.9611		BM25
2	0.1916		0.5957		BM25
3	0.1601		0.5494		BM25
4	0.0000		0.0000		동일
5	0.0000		0.0000		동일
6	0.0000		0.0000		동일
7	0.0000		0.0000		동일
8	0.1581		0.5957		BM25
9	0.0000		0.0000		동일
10	0.0000		0.0000		동일
11	0.1597		0.5957		BM25
12	0.0000		0.0000		동일
13	0.3838		1.9611		BM25
14	0.0000		0.0000		동일
15	0.2043		0.7166		BM25
16	0.2616		0.7166		BM25
17	0.0000		0.0000		동일
18	0.0000		0.0000		동일
19	0.0000		0.0000		동일
20	0.1861		0.6506		BM25
21	0.0000		0.0000		동일
22	0.0000		0.0000		동일
23	0.0000		0.0000		동일

=== 비교 분석 ===
TF-IDF: 코사인 유사도 기반, 벡터 공간에서의 각도 측정
BM25: 확률론적 랭킹 함수, 문서 길이와 단어 빈도 포화 고려
BM25가 일반적으로 검색 성능이 더 우수하다고 알려져 있습니다.



-----

### 4\. Word2Vec: 단어의 의미를 벡터에 담다

#### 💡 개념 (Concept)

BoW와 TF-IDF는 단어의 등장 빈도만 고려할 뿐, **단어의 의미나 문맥 정보**를 담지 못합니다. 예를 들어, "영화"와 "작품"은 의미가 유사하지만 DTM이나 TF-IDF 행렬에서는 완전히 다른 단어로 취급됩니다.

\*\*단어 임베딩(Word Embedding)\*\*은 이런 한계를 극복하기 위해 등장했습니다. 단어를 저차원(보통 100\~300차원)의 \*\*밀집 벡터(Dense Vector)\*\*로 표현하며, 이 벡터 공간 안에 단어의 의미와 문맥 정보를 압축하여 담아냅니다. **Word2Vec**은 가장 대표적인 단어 임베딩 모델로, "비슷한 문맥에서 등장하는 단어는 비슷한 의미를 가진다"는 **분포 가설**을 기반으로 합니다.

  - **CBOW (Continuous Bag-of-Words)**: 주변 단어들을 이용해 중심 단어를 예측합니다. (`[___]가 방에 들어간다` -\> `아버지`)
  - **Skip-gram**: 중심 단어를 이용해 주변 단어들을 예측합니다. (`아버지` -\> `[___]가 방에 들어간다`)

일반적으로 Skip-gram 방식이 더 많은 학습을 수행하여 희귀 단어나 데이터가 방대할 때 성능이 더 좋다고 알려져 있습니다.

#### 💻 예시 코드 (Example Code)

Word2Vec 모델을 학습시키려면 '토큰화된 문장들의 리스트' 형태의 데이터가 필요합니다. `gensim` 라이브러리를 사용하여 Word2Vec 모델을 학습하고 활용해 보겠습니다.

In [25]:
!pip install gensim


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from kiwipiepy import Kiwi
from gensim.models import Word2Vec

# 1. 모델 학습을 위한 데이터 준비 (토큰화된 문장 리스트)
# 실제 프로젝트에서는 대용량의 텍스트 데이터가 필요합니다.
# corpus = [
#     '배우의 연기력이 정말 대단한 영화였어요',
#     '스토리가 너무 예측 가능해서 연기력이 아까웠다',
#     '감독의 연출과 배우의 연기가 조화로웠던 영화',
#     '이 영화의 배우들은 연기를 정말 잘한다',
#     '스토리 구성이 탄탄해서 좋았던 작품이다',
#     '연출이 아쉬웠지만 배우들의 연기는 최고였다',
# ]

kiwi = Kiwi()
# 명사, 동사, 형용사만 추출
tokenized_corpus = [
    [token.form for token in kiwi.tokenize(doc) if token.tag in ['NNG', 'NNP', 'VV', 'VA']]
    for doc in corpus
]
print("토큰화된 데이터 (일부):", tokenized_corpus[0])

토큰화된 데이터 (일부): ['배우', '연기력', '영화']


In [27]:
# 2. Word2Vec 모델 학습
# vector_size: 임베딩 벡터의 차원
# window: 학습 시 고려할 주변 단어의 개수
# min_count: 학습에 사용할 단어의 최소 빈도
# sg=1: Skip-gram 방식 사용 (0은 CBOW)
model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=3, min_count=1, sg=1)

In [28]:
# 3. 학습된 임베딩 활용
# '배우'와 가장 유사한 단어 찾기
similar_words = model.wv.most_similar('배우', topn=3)
print("'배우'와 가장 유사한 단어:", similar_words)

'배우'와 가장 유사한 단어: [('처음', 0.21943432092666626), ('예측', 0.2172180563211441), ('몰입도', 0.20451846718788147)]


In [29]:
# 두 단어의 유사도 계산
similarity = model.wv.similarity('연기', '스토리')
print(f"'연기'와 '스토리'의 유사도: {similarity:.4f}")

'연기'와 '스토리'의 유사도: 0.0683


In [30]:
# 단어의 임베딩 벡터 확인
vector_actor = model.wv['배우']
print("'배우'의 임베딩 벡터 (처음 10개 값):", vector_actor[:10])

'배우'의 임베딩 벡터 (처음 10개 값): [-0.00054069  0.0002272   0.00510507  0.00902314 -0.0093082  -0.0071166
  0.00646014  0.00897839 -0.00502927 -0.0037563 ]


#### ✏️ 연습 문제 (Practice Problems)

1.  Word2Vec 모델을 학습시킬 때, `vector_size`를 200으로, `window`를 5로 변경하여 새로운 모델을 만들고, '영화'와 가장 유사한 단어를 찾아보세요. 결과가 어떻게 달라지는지 확인해 보세요.


In [40]:
# 코드 작성
model2 = Word2Vec(sentences=tokenized_corpus, vector_size=200, window=5, min_count=1, sg=1)

model2.wv.most_similar('영화', topn=1)

[('조화', 0.16198068857192993)]

2.  학습된 모델을 사용하여 '연기'와 '연출'의 유사도, '연기'와 '스토리'의 유사도를 각각 계산하고 비교 분석해 보세요.

In [39]:
# 코드 작성
similarity2 = model2.wv.similarity('연기', '연출')
print(similarity2)
similarity2 = model2.wv.similarity('연기', '스토리')
print(similarity2)

-0.029197108
0.06514456



-----

### 4\. FastText: OOV 문제를 해결하다

#### 💡 개념 (Concept)

Word2Vec은 강력하지만, 학습 데이터에 등장하지 않은 단어, 즉 **OOV(Out-of-Vocabulary)** 단어에 대해서는 임베딩 벡터를 생성할 수 없다는 치명적인 단점이 있습니다.

**FastText**는 Word2Vec의 Skip-gram 모델을 확장하여 이 문제를 해결했습니다. FastText의 핵심은 단어를 통째로 하나의 단위로 보지 않고, \*\*문자 단위의 n-gram(character n-gram)\*\*으로 세분화하여 보는 것입니다. 예를 들어, '영화'라는 단어를 3-gram으로 분해하면 `<영, 영화, 화>` 와 같은 부분 문자열들의 집합으로 표현됩니다. (여기서 `<, >`는 단어의 시작과 끝을 알리는 특수 문자입니다.)

이러한 접근 방식 덕분에, 학습 시 보지 못한 '감독님'이라는 OOV 단어가 등장해도, 그 단어를 구성하는 '감독', '독님', '님\>' 등의 부분 문자열 벡터들의 합으로 '감독님'의 전체 벡터를 추정할 수 있습니다. 이는 특히 신조어나 오탈자가 많은 한국어 처리에 매우 유용합니다.

#### 💻 예시 코드 (Example Code)

`gensim`의 `FastText` 모델은 `Word2Vec`과 사용법이 거의 동일합니다.

In [42]:
from kiwipiepy import Kiwi
from gensim.models import FastText

# 1. 모델 학습을 위한 데이터 준비 (위의 코퍼스 사용)
kiwi = Kiwi()
tokenized_corpus = [
    [token.form for token in kiwi.tokenize(doc) if token.tag in ['NNG', 'NNP', 'VV', 'VA']]
    for doc in corpus
]

In [43]:
# 2. FastText 모델 학습 (Word2Vec과 파라미터 동일)
ft_model = FastText(sentences=tokenized_corpus, vector_size=100, window=3, min_count=1, sg=1)

In [44]:
# 3. OOV 단어 테스트
oov_word = '연출가' # '연출'은 사전에 있지만 '연출가'는 코퍼스와 사전에 존재하지 않음

# FastText는 OOV 단어의 벡터를 생성할 수 있음
oov_vector = ft_model.wv[oov_word]
print(f"FastText: OOV 단어 '{oov_word}'의 벡터를 성공적으로 생성했습니다.")

FastText: OOV 단어 '연출가'의 벡터를 성공적으로 생성했습니다.


In [45]:
# 생성된 OOV 벡터로 유사 단어 찾기
similar_to_oov = ft_model.wv.most_similar(oov_word)
print(f"\n'{oov_word}'와 유사한 단어들: {similar_to_oov}")

# Word2Vec 모델(이전 섹션에서 생성)과 비교
# model.wv[oov_word] # -> 이 코드는 KeyError를 발생시킴


'연출가'와 유사한 단어들: [('연출', 0.18917475640773773), ('예측', 0.18507592380046844), ('괜찮', 0.18468232452869415), ('전체', 0.16199682652950287), ('연기력', 0.16190283000469208), ('대단하', 0.16189855337142944), ('센스', 0.1608278453350067), ('쓰', 0.1471981406211853), ('액션', 0.13927501440048218), ('감동', 0.13710856437683105)]


#### ✏️ 연습 문제 (Practice Problems)

1.  `FastText` 모델을 사용하여, 학습 말뭉치에 없을 법한 단어(예: '시나리오', '미장센' 등)의 임베딩 벡터가 생성되는지 확인하고, 해당 단어와 가장 유사한 단어들을 찾아보세요.


In [55]:
ft_model.wv['시나리오'][:10], ft_model.wv['미장센'][:10]

(array([-0.00272234, -0.00220198, -0.00019766, -0.00074125,  0.00153396,
        -0.00296082, -0.00292558, -0.00123987, -0.00116298,  0.00096288],
       dtype=float32),
 array([-1.9242261e-03, -1.9229419e-03, -4.5885518e-03, -4.1210265e-03,
        -2.1577997e-03,  9.7840931e-04, -5.9841590e-05, -2.8909277e-04,
         1.0450549e-03,  8.6090917e-04], dtype=float32))

2.  위에서 학습한 `Word2Vec` 모델과 `FastText` 모델에서 각각 '스토리'와 가장 유사한 단어를 찾아보고, 그 결과가 어떻게 다른지 비교해 보세요. `FastText`가 더 나은 결과를 보이는 경향이 있다면 그 이유는 무엇일지 설명해 보세요.

In [56]:
model.wv.most_similar('스토리')[0], ft_model.wv.most_similar('스토리')[0]

(('만들', 0.17862656712532043), ('연기', 0.18278983235359192))